# 2.2 Leaky integrate-and-fire モデル
## 2.2.1 LIFモデルの実装
生理学的なイオンチャネルの挙動は考慮せず, 入力電流を膜電位が閾値に達するまで時間的に積分するというモデルを\textbf{Integrate-and-fire (IF, 積分発火)モデル}といいます. さらに, IFモデルにおいて膜電位の漏れ(leak)\footnote{この漏れはイオンの拡散などによるものです. }も考慮したモデルを\textbf{Leaky integrate-and-fire (LIF, 漏れ積分発火) モデル}と呼びます. ここではLIFモデルのみを取り扱います. \par
ニューロンの膜電位を$V_m(t)$, 静止膜電位を$V_\text{rest}$, 入力電流\footnote{シナプス入力による電流がどうなるかは、シナプスのモデルの項で扱います。}を$I(t)$, 膜抵抗を$R_m$, 膜電位の時定数を$\tau_m\ (=R_m \cdot C_m)$とすると, 式は次のようになります\footnote{$(V_{m}(t)-V_\text{rest})$の部分は膜電位の基準を静止膜電位としたことにして, 単に$V_m(t)$だけの場合もあります. また, 右辺の$RI(t)$の部分は単に$I(t)$とされることもあります. 同じ表記ですが, この場合の$I(t)$はシナプス電流に比例する量, となっています(単位はmV). }. 
\begin{equation}
\tau_m \frac{dV_{m}(t)}{dt}=-(V_{m}(t)-V_\text{rest})+R_mI(t)
\end{equation}
ここで, $V_m$が閾値(threshold)\footnote{thから始まるので文字$\theta$が使われることもあります. }$V_{\text{th}}$を超えると, 脱分極が起こり, 膜電位はピーク電位 $V_{\text{peak}}$まで上昇します. 発火後は再分極が起こり, 膜電位はリセット電位 $V_{\text{reset}}$まで低下するとします\footnote{リセット電位は静止膜電位と同じ場合もあれば, 過分極を考慮して静止膜電位より低めに設定することもあります. }. 発火後, 一定の期間$\tau_{\text{ref}}$ の間は膜電位が変化しない\footnote{実装によっては不応期の間は膜電位の変化は許容するが発火は生じないようにすることもあります. }, とします(これを\textbf{不応期(refractory time
period)}と呼びます). 

以上を踏まえてLIFモデルを実装してみましょう. 簡単のために、まずはクラスを用いずに1個のニューロンについてのシミュレーションをしてみます\footnote{コードは\texttt{./SingleFileSimulations/Neurons/LIF\_single.py}です。}。 まずは定数を定義します。なお、時間の単位は秒に統一していますが、ミリ秒でも同じです。その場合は時定数などを1000倍してください(後に紹介するIzhikevich モデルではミリ秒の単位を用いるので注意してください)\footnote{これは個人的な慣れによります。}。

In [ ]:
dt = 5e-5; T = 0.4 # (s)
nt = round(T/dt) # シミュレーションのステップ数
 
tref = 2e-3 # 不応期 (s)
tc_m = 1e-2 # 膜時定数 (s)
vrest = -60 # 静止膜電位 (mV) 
vreset = -65 # リセット電位 (mV) 
vthr = -40 # 閾値電位 (mV)
vpeak = 30 #　ピーク電位 (mV)
  
t = np.arange(nt)*dt*1e3 # 時間(ms)
I = 25*(t>50) - 25*(t>350) # 入力電流(に比例する値)(mV)

# 初期化
v = vreset # 膜電位の初期値
tlast = 0 # 最後のスパイクの時間を記録する変数 
v_arr = np.zeros(nt) # 膜電位を記録する配列

\texttt{I}はHHモデルのときと同じように矩形波を用いてパルス入力をしています。また、入力電流ではなく入力電流に比例する量となっていますが、これは膜抵抗を乗じた後の値であると考えてください。それではメインとなる部分を書いてみましょう。

In [ ]:
for i in tqdm(range(nt)):
    dv = (vrest - v + I[i]) / tc_m # 膜電位の変化量
    v = v + ((dt*i) > (tlast + tref))*dv*dt # 更新
    
    s = 1*(v>=vthr) # 発火の確認
    tlast = tlast*(1-s) + dt*i*s # 発火時刻の更新
    v = v*(1-s) + vpeak*s # 発火している場合ピーク電位に更新
    v_arr[i] = v  # 膜電位の値を保存
    v = v*(1-s) + vreset*s # 発火している場合に膜電位をリセット

if文を使って書かれることが多い部分を、if文を使用せずに書いています\footnote{これはPythonがforループを用いると遅いことによります。if文を用いずにベクトルの計算で発火時の動態を記述することで「Pythonにおいては」高速に処理ができます。}。まず、
\texttt{((dt*i) > (tlast + tref))} の部分は最後に発火した時刻(\texttt{tlast})に不応期(\texttt{tref})を足した時刻を現在の時刻(\texttt{dt*i})が上回っていれば、\texttt{v}に\texttt{dv*dt}を加算する、という意味です。次に\texttt{(v>=vthr)}はBoolean型となり、膜電位が閾値\texttt{vthr}を超えていれば\texttt{True}, そうでなければ\texttt{False}となります。
これに1を乗じることでint型となります\footnote{他には0を加算することでもBoolean型からint型への変換が可能です。\texttt{v}が配列ではなく、1変数の場合は\colorbox{shadecolor}{\texttt{s = 1 if (v>=vthr) else 0}}
とする方が高速です。}(1が発火、0が未発火)。その下は得られたスパイクの変数\texttt{s}を用いて、値の更新を行っています。
閾値を超えなかった場合(\texttt{s=0})は\texttt{(1-s)}が乗じられた値となり、閾値を超えた場合(\texttt{s=1})は\texttt{s}が乗じられた値となります。\par
最後に\texttt{v\_arr}に保存された膜電位変化を描画してみましょう。

In [ ]:
plt.figure(figsize=(5, 3))
plt.plot(t, v_arr)
plt.xlim(0, t.max())
plt.xlabel('Time (ms)'); plt.ylabel('Membrane potential (mV)') 
plt.show()

LIFモデルの膜電位変化。50 msから350 msまで電流を印加している。

## 2.2.2 LIFモデルのF-I curve
### 数値的計算によるF-I curveの描画
この節ではLIFモデルにおける入力電流に対する発火率の変化がどのようになるかを考えます。次のコードのように入力電流を徐々に増加させたときの発火率を見てみましょう

In [ ]:
dt = 5e-5; T = 1; nt = round(T/dt)
tref = 5e-3; tc_m = 1e-2; vrest = 0; vreset = 0; vthr = 1

I_max = 3 # (nA)
N = 100 # N種類の入力電流
I = np.linspace(0, I_max, N) # 入力電流(pA)
spikes = np.zeros((N, nt)) # スパイクの記録変数

for i in tqdm(range(N)):
    v = vreset; tlast = 0 # 初期化
    for t in range(nt):
        dv = (vrest - v + I[i]) / tc_m #　膜電位の導関数
        update = 1 if ((dt*t) > (tlast + tref)) else 0 # 不応期でないかの確認
        v = v + update*dv*dt # 膜電位の更新
        s = 1 if (v>=vthr) else 0 #発火時は1, その他は0の出力
        tlast = tlast*(1-s) + dt*t*s # スパイク時刻の更新
        spikes[i, t] = s # 保存
        v = v*(1-s) + vreset*s # 膜電位のリセット

# 描画
rate = np.sum(spikes, axis=1) / T # 発火率
plt.figure(figsize=(4, 3))
plt.plot(I, rate)
plt.xlabel('Input current (nA)')
plt.ylabel('Firing rate (Hz)') 
plt.show()

結果は図\ref{fig:fi_n}のようになります。このような曲線を\textbf{frequency-current (F-I) curve} (または neuronal input/output (I/O) curve)と呼びます.

\caption{LIFニューロンの入力電流に対する発火率の関係(F-I curve)を数値的に求めた結果. (左)不応期がない場合。この場合は閾値付近以外はReLU関数のような挙動をします。また実際にこのような直線的なF-I curveを持つニューロンもあります(関連した考察は発火率モデルの項で行います)。(右)不応期を5 msとした場合。さらに電流を強めると発火率は飽和(saturation)します。}

### 解析的計算によるF-I curveの描画
ここまでは数値的なシミュレーションによりF-I curveを求めましたが、以下では解析的にF-I curveの式を求めてみましょう。具体的には、一定かつ持続的な入力電流を$I$としたときのLIFニューロンの発火率(firing rate)が

$$
\begin{equation}
\text{rate}\sim \left(\tau_m \ln \dfrac{R_mI}{R_mI-V_{\text{th}}}\right)    
\end{equation}
$$

と近似できることを示します. まず、$t=t_1$にスパイクが生じたとします. このとき, 膜電位はリセットされるので$V_m(t_1)=0$です. $[t_1, t]$における膜電位はLIFの式を積分することで得られます.

$$
\begin{equation}
\tau \frac{dV_{m}(t)}{dt}=-V_{m}(t)+R I(t)
\end{equation}
$$

の式を積分すると, 

$$
\begin{aligned}
\int_{t_1}^{t} \frac{\tau dV_m}{RI-V_m}&=\int_{t_1}^{t} dt\\
\ln(1-\frac{V_m(t)}{RI})&=-\frac{t-t_1}{\tau} \ \ \ (\because V_m(t_1)=0)\\
\therefore\ \ V_m(t) &=RI\left[1-\exp\left(-\frac{t-t_1}{\tau}\right)\right] 
\end{aligned}
$$

となります. $t>t_1$における初めのスパイクが$t=t_2$に生じたとすると, そのときの膜電位は$V_m(t_2)=V_{\text{th}}$です（実際には閾値以上となっている場合もあるますが近似します）. $t=t_2$を上の式に代入して

$$
\begin{align}
V_{\text{th}}&=RI\left[1-\exp\left(-\frac{t_2-t_1}{\tau}\right)\right] \\
\therefore\ \ T&= t_2-t_1 =  \tau \ln \frac{RI}{RI-V_{\text{th}}}
\end{align}
$$

となります. ここで$T$は2つのスパイクの時間間隔です. $t_1\leq t<t_2$におけるスパイクは$t=t_1$時の1つなので, 発火率は$1/T$となります. よって

$$
\text{rate}\sim \left(\tau \ln \frac{RI}{RI-V_{\text{th}}}\right)
$$

です. 不応期$\tau_{\text{ref}}$を考慮すると, 持続的に入力がある場合は単純に$\tau_{\text{ref}}$だけ発火が遅れるので発火率は$1/(\tau_{\text{ref}}+T)$となります. 

それでは式(1.9)に基づいてF-I curveを描画してみましょう。

In [ ]:
tc_m = 1e-2 # 膜時定数 (s)
R = 1 #膜抵抗 
vthr = 1 # 閾値電位 (mV)
tref = 5e-3 # 不応期 (s)
I_max = 3 # 最大電流
I = np.arange(0, I_max, 0.01) #入力電流

rate = 1 / (tref + tc_m*np.log(R*I / (R*I - vthr)))
rate[np.isnan(rate)] = 0 # nan to 0

# 描画
plt.figure(figsize=(4, 3))
plt.plot(I, rate, color="k")
plt.xlabel('Input current (nA)'); plt.ylabel('Firing rate (Hz)') 
plt.xlim(0, I_max)
plt.show()

閾値以下の場合は$\log$の真数が負になるので\texttt{RuntimeWarning}がでますが気にせずに実行しましょう。結果は図のようになります。

\caption{LIFニューロンの入力電流に対する発火率の関係(F-I curve)を解析的に求めた結果. (左)不応期がない場合。(右)不応期を5 msとした場合。}
\label{fig:fi_n}